# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4896, uptime 0:00:35
mini_beamline                    RUNNING   pid 4897, uptime 0:00:35
random_walk                      RUNNING   pid 4912, uptime 0:00:32
random_walk_horiz                RUNNING   pid 4899, uptime 0:00:35
random_walk_vert                 RUNNING   pid 4900, uptime 0:00:35
simple                           RUNNING   pid 4901, uptime 0:00:35
thermo_sim                       RUNNING   pid 4902, uptime 0:00:35
trigger_with_pc                  RUNNING   pid 4903, uptime 0:00:35


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-24 20:15:46
Persistent Unique Scan ID: '6dba2cbe-46e7-446d-9fb1-d70715e8fb0f'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:15:46.3 |     -1.000 |      94974 |
|         2 | 20:15:46.4 |     -0.500 |      95343 |
|         3 | 20:15:46.4 |      0.000 |      95773 |
|         4 | 20:15:46.4 |      0.500 |      94776 |
|         5 | 20:15:46.4 |      1.000 |      93621 |
+-----------+------------+------------+------------+
generator scan ['6dba2cbe'] (scan num: 3)





('6dba2cbe-46e7-446d-9fb1-d70715e8fb0f',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/6dba2cbe-46e7-446d-9fb1-d70715e8fb0f-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-24 20:15:46
Persistent Unique Scan ID: '76666872-2e72-460a-a66a-4d42813f7b0f'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:15:46.9 |          -1 | 1053271334 |


|         2 | 20:15:47.0 |          -0 | 1056005817 |


|         3 | 20:15:47.2 |           0 | 1059370573 |


|         4 | 20:15:47.3 |           0 | 1064004360 |


|         5 | 20:15:47.5 |           1 | 1072643707 |


+-----------+------------+-------------+------------+
generator scan ['76666872'] (scan num: 4)





('76666872-2e72-460a-a66a-4d42813f7b0f',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-24 20:15:47
Persistent Unique Scan ID: '00e21502-ad43-47e5-8ba0-6f63522fb373'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:15:48.1 | 1103647389 |


|         2 | 20:15:49.0 | 1130738436 |


|         3 | 20:15:50.0 | 1083700589 |


...

|        28 | 20:16:15.0 | 1056974656 |


|        29 | 20:16:16.0 | 1103674188 |


|        30 | 20:16:17.0 | 1128235548 |


+-----------+------------+------------+
generator count ['00e21502'] (scan num: 5)





('00e21502-ad43-47e5-8ba0-6f63522fb373',)